# Import libraries

In [13]:
# !pip install -U tqdm
# !pip install textstat

     |████████████████████████████████| 3.0MB 4.2MB/s 


In [79]:
import pandas as pd
import numpy as np

import gc
import time

from tqdm import tqdm
tqdm.pandas()

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')

from gensim.summarization.summarizer import summarize

import textstat

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Download dataset

In [2]:
!wget 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_US_v1_00.tsv.gz'

--2020-01-15 17:51:40--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_US_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1466965039 (1.4G) [application/x-gzip]
Saving to: ‘amazon_reviews_multilingual_US_v1_00.tsv.gz’

amazon_reviews_mult 100%[===================>]   1.37G  83.1MB/s    in 18s     

2020-01-15 17:51:57 (79.8 MB/s) - ‘amazon_reviews_multilingual_US_v1_00.tsv.gz’ saved [1466965039/1466965039]



In [0]:
!gunzip amazon_reviews_multilingual_US_v1_00.tsv.gz

# Data cleaning

In [2]:
df = pd.read_csv('amazon_reviews_multilingual_US_v1_00.tsv', delimiter='\t', error_bad_lines=False)
df

b'Skipping line 3231472: expected 15 fields, saw 22\n'
b'Skipping line 3509762: expected 15 fields, saw 22\n'
b'Skipping line 4018793: expected 15 fields, saw 22\n'
b'Skipping line 4280173: expected 15 fields, saw 22\nSkipping line 4290596: expected 15 fields, saw 22\n'
b'Skipping line 4331421: expected 15 fields, saw 22\nSkipping line 4340267: expected 15 fields, saw 22\nSkipping line 4341665: expected 15 fields, saw 22\nSkipping line 4386155: expected 15 fields, saw 22\nSkipping line 4388098: expected 15 fields, saw 22\n'
b'Skipping line 4408027: expected 15 fields, saw 22\nSkipping line 4442615: expected 15 fields, saw 22\n'
b'Skipping line 4519623: expected 15 fields, saw 22\n'
b'Skipping line 4525797: expected 15 fields, saw 22\nSkipping line 4543519: expected 15 fields, saw 22\n'
b'Skipping line 4587726: expected 15 fields, saw 22\nSkipping line 4589301: expected 15 fields, saw 22\nSkipping line 4634393: expected 15 fields, saw 22\n'
b'Skipping line 4666168: expected 15 fields, s

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,53096384,R63J84G1LOX6R,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,4.0,0.0,1.0,N,N,ignore the review below,this is the first 8 issues of the series. it i...,1995-08-13
1,US,53096399,R1BALOA11Z06MT,1559947608,381720534,The 22 Immutable Laws of Marketing,Books,4.0,0.0,0.0,N,N,awesome,I've always been partial to immutable laws. Th...,1995-08-17
2,US,53096332,R1LLAY5W5PZUS4,0671701800,860650224,Contact,Books,5.0,1.0,2.0,N,N,Read the book. It's good.,This is a book about first contact with aliens...,1995-08-30
3,US,53096335,R3R9VTJ82FXECQ,0425132153,624269601,Good Omens,Books,5.0,0.0,0.0,N,N,Funniest book ever written about the Anti-Christ,This is quite possibly *the* funniest book I h...,1995-09-11
4,US,51747709,R1P5J3FNBWTFXY,0517122707,161411385,A Confederacy of Dunces,Books,5.0,0.0,0.0,N,N,A winner that didn't last. Only a mothers love...,The story behind the book is almost better tha...,1995-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6900881,US,52303,R16DAHIWNXFOYI,B00QW8TYWO,828652896,Crossy Road,Mobile_Apps,5.0,0.0,0.0,N,Y,Five Stars,very fun game,2015-08-31
6900882,US,565563,R1DS6EBGAEJDB5,B00PSLQYWE,156264553,Downton Abbey Season 5,Digital_Video_Download,5.0,0.0,0.0,N,Y,Five Stars,Great,2015-08-31
6900883,US,254421,R3HEHT8JUA33VW,B000FC1LOC,354573823,My Sister's Keeper: A Novel (Wsp Readers Club),Digital_Ebook_Purchase,5.0,0.0,0.0,N,N,and she has beautifully written and explored t...,"This is my first book from Jodi, and she has b...",2015-08-31
6900884,US,146004,RVK4I33L93536,B008T4HHWS,599809487,Scent of a Woman,Digital_Video_Download,5.0,0.0,0.0,N,N,Like this!,Good movie!,2015-08-31


In [0]:
df.drop(columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'vine', 'review_date'], inplace=True)

In [4]:
# dropping rows with less than 20 words in review_body
print(df.shape)

MIN_WORDS = 25
df['review_body_length'] = df['review_body'].progress_map(lambda x: len(str(x).split()))
df = df[df['review_body_length'] >= MIN_WORDS]
MIN_SENTENCES = 3
df['review_body_sentences'] = df['review_body'].progress_map(lambda x: textstat.sentence_count(str(x)))
df = df[df['review_body_sentences'] >= MIN_SENTENCES]

print(df.shape)

  0%|          | 1/6900886 [00:00<291:04:41,  6.59it/s]

(6900886, 8)


100%|██████████| 4023562/4023562 [07:19<00:00, 9159.28it/s] 
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


(1957608, 10)


In [0]:
# dropping rows with no verified_purchase 
df = df[df['verified_purchase'] == 'Y']

# dropping rows with no helpful ratings
# df = df[df['helpful_votes'] >= 1.0]

In [9]:
# Keeping rows with category frequency above certain threshold
MIN_FREQ = 15000
allowed_categories = df['product_category'].value_counts().loc[lambda x: x >= MIN_FREQ]
print(allowed_categories)
allowed_categories = allowed_categories.index.tolist()

df = df[df['product_category'].isin(allowed_categories)]
print(df.shape)

Digital_Ebook_Purchase    280127
Mobile_Apps               254628
Video DVD                 156855
Books                     134883
Digital_Video_Download    116103
Music                      84559
Digital_Music_Purchase     19977
Name: product_category, dtype: int64
(1047132, 10)


In [10]:
def combine_categories(category):
  if category in ['Video DVD', 'Digital_Video_Download', 'Video']:
    return 'Films_TV'
  elif category in ['Music', 'Digital_Music_Purchase']:
    return 'Music'
  elif category in ['Digital_Ebook_Purchase', 'Books']:
    return 'Books'
  else:
    return category
  
df['product_category'] = df['product_category'].progress_map(combine_categories)
df['product_category'].value_counts()

100%|██████████| 1047132/1047132 [00:01<00:00, 997733.95it/s] 


Books          415010
Films_TV       272958
Mobile_Apps    254628
Music          104536
Name: product_category, dtype: int64

In [11]:
def get_sentiment(rating):
  if rating > 3:
    return 'Positive'
  elif rating < 3:
    return 'Negative'
  else:
    return 'Neutral'
  
df['sentiment'] = df['star_rating'].progress_map(get_sentiment)

100%|██████████| 1047132/1047132 [00:00<00:00, 1048029.78it/s]


In [0]:
df['review'] = df[['review_headline', 'review_body']].astype(str).apply('.'.join, axis=1)
# df.drop(columns = ['review_headline', 'review_body'], inplace=True)

In [13]:
df.reset_index(drop=True, inplace=True)
df

,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_body_length,review_body_sentences,sentiment,review
0,Ender's Game (The Ender Quintet),Books,5.0,0.0,0.0,Y,HOW COULD ANYONE ONLY RATE THIS NOVEL A SIX?!?,Ender's Game is the most riveting and moving s...,80,4,Positive,HOW COULD ANYONE ONLY RATE THIS NOVEL A SIX?!?...
1,Drums of Autumn (Outlander),Books,5.0,0.0,0.0,Y,Great!! Can't wait for Drums of Autumn,I have read the other 3 but can't wait for the...,98,4,Positive,Great!! Can't wait for Drums of Autumn.I have ...
2,I Am That: Talks with Sri Nisargadatta Maharaj,Books,5.0,6.0,6.0,Y,Find out the answer of all Questions.,This book can show you the way of how to put a...,64,5,Positive,Find out the answer of all Questions..This boo...
3,The Millionaire Next Door: The Surprising Secr...,Books,4.0,0.0,1.0,Y,"Nice work, but let's hear more...","I firmly believe in the principles, having see...",196,5,Positive,"Nice work, but let's hear more....I firmly bel..."
4,The Day After Roswell,Books,5.0,3.0,4.0,Y,Corso's claims may still not satisfy debunkers,There are two credible eye witnesses to Roswel...,450,5,Positive,Corso's claims may still not satisfy debunkers...
...,...,...,...,...,...,...,...,...,...,...,...,...
1047127,Erased,Films_TV,4.0,0.0,0.0,Y,I swear I still enjoyed the movie,the difference here is the style. So its a lit...,319,14,Positive,I swear I still enjoyed the movie.the differen...
1047128,The Picture of Dorian Gray,Books,5.0,0.0,0.0,Y,Don't miss this one!,Oscar Wilde is a master of the epigram - almos...,78,4,Positive,Don't miss this one!.Oscar Wilde is a master o...
1047129,To Kill A Mockingbird,Books,5.0,0.0,0.0,Y,A Genuine Classic,What's there not to love about this book? The ...,40,5,Positive,A Genuine Classic.What's there not to love abo...
1047130,Man of Steel (2013),Films_TV,2.0,1.0,4.0,Y,I just appreciate Chris Nolan and was looking ...,I did not grow up watching Superman so the ori...,52,5,Negative,I just appreciate Chris Nolan and was looking ...


# Text summarization

In [14]:
def summarize_text(review):
  try:
    return summarize(review)
  except ValueError:
    return None
  
df['summarized_review'] = df['review'].progress_map(summarize_text)
df

100%|██████████| 1047132/1047132 [35:33<00:00, 490.81it/s]


,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_body_length,review_body_sentences,sentiment,review,summarized_review
0,Ender's Game (The Ender Quintet),Books,5.0,0.0,0.0,Y,HOW COULD ANYONE ONLY RATE THIS NOVEL A SIX?!?,Ender's Game is the most riveting and moving s...,80,4,Positive,HOW COULD ANYONE ONLY RATE THIS NOVEL A SIX?!?...,
1,Drums of Autumn (Outlander),Books,5.0,0.0,0.0,Y,Great!! Can't wait for Drums of Autumn,I have read the other 3 but can't wait for the...,98,4,Positive,Great!! Can't wait for Drums of Autumn.I have ...,Can't wait for Drums of Autumn.I have read the...
2,I Am That: Talks with Sri Nisargadatta Maharaj,Books,5.0,6.0,6.0,Y,Find out the answer of all Questions.,This book can show you the way of how to put a...,64,5,Positive,Find out the answer of all Questions..This boo...,It can add a lot on your thinking way of under...
3,The Millionaire Next Door: The Surprising Secr...,Books,4.0,0.0,1.0,Y,"Nice work, but let's hear more...","I firmly believe in the principles, having see...",196,5,Positive,"Nice work, but let's hear more....I firmly bel...",I live in an area that is like a case study fo...
4,The Day After Roswell,Books,5.0,3.0,4.0,Y,Corso's claims may still not satisfy debunkers,There are two credible eye witnesses to Roswel...,450,5,Positive,Corso's claims may still not satisfy debunkers...,Corso's claims may still not satisfy debunkers...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047127,Erased,Films_TV,4.0,0.0,0.0,Y,I swear I still enjoyed the movie,the difference here is the style. So its a lit...,319,14,Positive,I swear I still enjoyed the movie.the differen...,Other considerations for this movie are not so...
1047128,The Picture of Dorian Gray,Books,5.0,0.0,0.0,Y,Don't miss this one!,Oscar Wilde is a master of the epigram - almos...,78,4,Positive,Don't miss this one!.Oscar Wilde is a master o...,
1047129,To Kill A Mockingbird,Books,5.0,0.0,0.0,Y,A Genuine Classic,What's there not to love about this book? The ...,40,5,Positive,A Genuine Classic.What's there not to love abo...,
1047130,Man of Steel (2013),Films_TV,2.0,1.0,4.0,Y,I just appreciate Chris Nolan and was looking ...,I did not grow up watching Superman so the ori...,52,5,Negative,I just appreciate Chris Nolan and was looking ...,I just appreciate Chris Nolan and was looking ...


In [32]:
example_indexes = [660, 774, 896, 1086, 22269, 37481, 77494, 79958, 86978, 678099, 819676, 848417, 1016967, 1034908, 1045842, 1046297]
for item in example_indexes:
  current_row = df.iloc[item]
  print('Product title: {}\nProduct category: {}\nReview headline: {}\nReview body: {}\nReview summary: {}\n\n'.format(
      current_row['product_title'], current_row['product_category'],
      current_row['review_headline'], current_row['review_body'],
      current_row['summarized_review']
  ))

Product title: Angela's Ashes (The Frank McCourt Memoirs)
Product category: Books
Review headline: A version of &quot;Roots&quot; for the Irish
Review body: All of a sudden the actions of many of my relatives seemed part of a larger pattern. My God! Was it because we have Irish roots &amp; souls? I did NOT find this book to be &quot;self-pity tripe&quot; as others have described it. I thought the author spoke in a unique running narrative,  with a matter-of-fact rather than &quot;woe is me&quot; tone. Hell, I  challenge anyone describing this as their history to make it sound upbeat.  The truth was a story in itself and it has occured many times over  throughout our human history. That he survived in an indifferent world was  uplifting. I have many connections to this. I come from a long line of  passionate, creative people who love and live hard. Many of them have made  the mistakes of McCourt's father and mother. A handful have gone to succeed  like the author and his siblings.  But 

# Text preprocessing

In [0]:
ps = PorterStemmer()
stopwords_set = set(stopwords.words('english'))

In [34]:
def preprocess_text(review):
  review = re.sub('[^a-zA-Z]', ' ', review)
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords_set]
  review = ' '.join(review)
  return review

df['review'] = df['review'].progress_map(preprocess_text)

100%|██████████| 1047132/1047132 [17:39<00:00, 988.19it/s] 


# TF-IDF

In [0]:
# SPLIT TRAIN TEST

train, test = train_test_split(df, test_size=0.2, random_state = 42)

In [0]:
## TF-IDF

parser = TfidfVectorizer(min_df = 16)
# create the document term matrix
X_train = parser.fit_transform(train['review'])
X_test = parser.transform(test['review'])

In [52]:
print('Number of tokens: {}\nTF-IDF matrix shape: {}\nList of tokens: {}'.format(
    len(parser.get_feature_names()),
    X_train.shape,
    parser.get_feature_names()
))

Number of tokens: 26010
TF-IDF matrix shape: (837705, 26010)
List of tokens: ['aa', 'aaa', 'aaaaa', 'aac', 'aacut', 'aah', 'aand', 'aaron', 'aarp', 'ab', 'aback', 'abad', 'abandon', 'abat', 'abb', 'abba', 'abbado', 'abbey', 'abbi', 'abbott', 'abbottabad', 'abbrevi', 'abc', 'abd', 'abdi', 'abdic', 'abdomen', 'abdomin', 'abduct', 'abductor', 'abdul', 'abdullah', 'abe', 'abel', 'abercrombi', 'abernathi', 'aberr', 'abet', 'abhor', 'abhorr', 'abhorsen', 'abid', 'abigail', 'abil', 'abit', 'abject', 'abkco', 'abl', 'abli', 'ablum', 'abneg', 'abnorm', 'aboard', 'abod', 'abolish', 'abolit', 'abolitionist', 'abomin', 'aborigin', 'abort', 'abou', 'abound', 'abra', 'abraham', 'abram', 'abras', 'abraxa', 'abreast', 'abridg', 'abroad', 'abrupt', 'abruptli', 'absenc', 'absent', 'absente', 'absentia', 'absolut', 'absolutist', 'absolutley', 'absolutli', 'absolv', 'absorb', 'absorpt', 'absoult', 'absoulut', 'abstain', 'abstin', 'abstract', 'abstrus', 'absurd', 'absurdist', 'absurdli', 'abt', 'abu', 'abu

# Label classification

In [53]:
# Create the model with 1000 trees
category_model = RandomForestClassifier(n_estimators=512, 
                               max_depth = 32,
                               min_samples_leaf = 8,
                               max_features = 'sqrt',
                               verbose = 10,
                               n_jobs=-1)
# Fit on training data
category_model.fit(X_train, train['product_category'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 512building tree 2 of 512



[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.2s


building tree 3 of 512
building tree 4 of 512
building tree 5 of 512
building tree 6 of 512


[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   17.2s


...

building tree 479 of 512
building tree 480 of 512
building tree 481 of 512
building tree 482 of 512
building tree 483 of 512
building tree 484 of 512
building tree 485 of 512
building tree 486 of 512
building tree 487 of 512
building tree 488 of 512
building tree 489 of 512
building tree 490 of 512
building tree 491 of 512
building tree 492 of 512
building tree 493 of 512
building tree 494 of 512
building tree 495 of 512
building tree 496 of 512
building tree 497 of 512
building tree 498 of 512
building tree 499 of 512
building tree 500 of 512
building tree 501 of 512
building tree 502 of 512
building tree 503 of 512
building tree 504 of 512
building tree 505 of 512
building tree 506 of 512
building tree 507 of 512
building tree 508 of 512
building tree 509 of 512


[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed: 34.5min


building tree 510 of 512
building tree 511 of 512
building tree 512 of 512


[Parallel(n_jobs=-1)]: Done 512 out of 512 | elapsed: 34.7min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=32, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=8, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=512,
                       n_jobs=-1, oob_score=False, random_state=None,
                       verbose=10, warm_start=False)

In [54]:
# Prediction
rf_predictions = category_model.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    1.3s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.7s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.1s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    2.6s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    3.1s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    3.6s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    4.2s
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed:    4.9s
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed:    5.6s
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed:    6.3s
[Para

In [56]:
# Acuuracy

print(classification_report(test['product_category'], rf_predictions))

              precision    recall  f1-score   support

       Books       0.89      0.99      0.94     83020
    Films_TV       0.96      0.87      0.92     54503
 Mobile_Apps       0.98      0.96      0.97     50887
       Music       0.97      0.82      0.89     21017

    accuracy                           0.93    209427
   macro avg       0.95      0.91      0.93    209427
weighted avg       0.94      0.93      0.93    209427



In [57]:
test['product_category_prediction'] = rf_predictions

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Sentiment analysis

In [64]:
# Create the model with 500 trees
sentiment_analysis_model = LinearSVC(verbose=10)

# Fit on training data
sentiment_analysis_model.fit(X_train, train['sentiment'])

[LibLinear]

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=10)

In [0]:
# Prediction
svm_predictions = sentiment_analysis_model.predict(X_test)

In [66]:
print(classification_report(test['sentiment'], svm_predictions))

              precision    recall  f1-score   support

    Negative       0.71      0.65      0.68     21681
     Neutral       0.55      0.16      0.24     17630
    Positive       0.90      0.98      0.94    170116

    accuracy                           0.88    209427
   macro avg       0.72      0.60      0.62    209427
weighted avg       0.85      0.88      0.85    209427



In [67]:
test['sentiment_prediction'] = svm_predictions

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
test

,product_title,product_category,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,review_body_length,review_body_sentences,sentiment,review,summarized_review,product_category_prediction,sentiment_prediction
612910,Life of Pi (Blu-ray + DVD + Digital Copy),Films_TV,1.0,7.0,21.0,Y,Slow paced and BOOOOOOOORING,I shut off the DVD player after a half an hour...,71,7,Negative,slow pace boooooooor shut dvd player half hour...,Slow paced and BOOOOOOOORING.I shut off the DV...,Films_TV,Negative
975635,The Windup Girl,Books,2.0,5.0,6.0,Y,Not for Everyone,Nothing gets me more excited than rich world b...,306,10,Negative,everyon noth get excit rich world build noth p...,To me this book would take multiple read throu...,Books,Positive
494142,Cloud Atlas [Blu-ray],Films_TV,5.0,2.0,2.0,Y,Best movie ever,I first watched this through Netflix. It's rea...,144,12,Positive,best movi ever first watch netflix realli long...,Best movie ever.I first watched this through N...,Films_TV,Positive
675563,Babymetal,Music,5.0,6.0,6.0,Y,Exceeded expectations!! Shockingly good!!,Absolutely love this CD. I had seen the videos...,82,8,Positive,exceed expect shockingli good absolut love cd ...,"I had seen the videos, loved the songs, and fo...",Music,Positive
301516,Dredd,Films_TV,3.0,1.0,1.0,Y,Worth the Rental,Not bad action packed film. The movie had some...,38,4,Neutral,worth rental bad action pack film movi imagin ...,,Films_TV,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287053,"Catching Fire (Hunger Games Trilogy, Book 2)",Books,5.0,0.0,0.0,Y,Captivating!,I saw the movie of the first book. So I wanted...,48,5,Positive,captiv saw movi first book want read one saw m...,Captivating!.I saw the movie of the first book.,Books,Positive
595378,The Goldfinch: A Novel (Pulitzer Prize for Fic...,Books,5.0,1.0,1.0,Y,You Won't Forget This Novel,The Goldfinch should become an American classi...,53,5,Positive,forget novel goldfinch becom american classic ...,,Books,Positive
262848,Elf,Films_TV,5.0,0.0,1.0,Y,Great movie,"Just watch the movie, it's a lot of fun. Wil i...",26,3,Positive,great movi watch movi lot fun wil great kid en...,,Films_TV,Positive
388085,Goodnight Moon,Books,5.0,0.0,0.0,Y,Classic Nighttime Story,My girls both love this story so much. The way...,60,5,Positive,classic nighttim stori girl love stori much wa...,Classic Nighttime Story.My girls both love thi...,Books,Positive


# Topic modelling

In [0]:
tfidf_vect = TfidfVectorizer()
doc_term_matrix = tfidf_vect.fit_transform(df['review'])

In [83]:
# NMF

nmf = NMF(n_components=7, random_state=42, verbose=10)
nmf.fit(doc_term_matrix)

violation: 1.0
violation: 0.596763537965358
violation: 0.32411455396805716
violation: 0.16998861176587404
violation: 0.09725839698307767
violation: 0.05858580010889679
violation: 0.036744845364467996
violation: 0.0238515743287151
violation: 0.01586250964658752
violation: 0.010964971220597624
violation: 0.007909770382010543
violation: 0.005916974893884169
violation: 0.004560035048940126
violation: 0.003605010848880966
violation: 0.002916963284380271
violation: 0.002400393592371093
violation: 0.0019951358567646517
violation: 0.0016765966163552143
violation: 0.0014212351443085248
violation: 0.001212024635305093
violation: 0.0010375585043973681
violation: 0.0008907344508357652
violation: 0.0007667283118024481
violation: 0.0006612842494809855
violation: 0.0005711394934757408
violation: 0.0004938377694634839
violation: 0.00042747408887506416
violation: 0.00037023065450237405
violation: 0.0003207681921424787
violation: 0.00027797742262149775
violation: 0.0002409269249169592
violation: 0.00020

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=10)

In [84]:
for i, topic in enumerate(nmf.components_):
  print(f'Top 10 words for topic #{i}:')
  print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
  print('\n')

Top 10 words for topic #0:
['seri', 'great', 'put', 'one', 'good', 'charact', 'end', 'stori', 'read', 'book']


Top 10 words for topic #1:
['realli', 'great', 'level', 'challeng', 'like', 'get', 'addict', 'play', 'fun', 'game']


Top 10 words for topic #2:
['enjoy', 'like', 'time', 'one', 'see', 'film', 'good', 'great', 'watch', 'movi']


Top 10 words for topic #3:
['peopl', 'thing', 'get', 'stori', 'one', 'make', 'like', 'charact', 'film', 'br']


Top 10 words for topic #4:
['band', 'like', 'great', 'one', 'sound', 'listen', 'cd', 'music', 'song', 'album']


Top 10 words for topic #5:
['updat', 'get', 'download', 'great', 'free', 'fire', 'work', 'kindl', 'use', 'app']


Top 10 words for topic #6:
['absolut', 'watch', 'wait', 'amaz', 'awesom', 'stori', 'season', 'show', 'seri', 'love']




In [85]:
topic_values = nmf.transform(doc_term_matrix)
df['topic'] = topic_values.argmax(axis=1)

violation: 1.0
violation: 0.2513833888462288
violation: 0.014008312295587952
violation: 0.0027801374235043156
violation: 0.00038717368481803647
violation: 5.9097241149836855e-05
Converged at iteration 6
